NAME : Prathmesh Patil

Class : Btech (A)

PRN : 23SC114502026

In [ ]:
pip install fairlearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 10.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from fairlearn.metrics import MetricFrame, true_positive_rate, false_positive_rate

In [ ]:
!wget https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv

--2025-08-18 06:13:12--  https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2546489 (2.4M) [text/plain]
Saving to: ‘compas-scores-two-years.csv’

compas-scores-two-y 100%[===================>]   2.43M  --.-KB/s    in 0.01s   

2025-08-18 06:13:13 (208 MB/s) - ‘compas-scores-two-years.csv’ saved [2546489/2546489]



In [ ]:
df = pd.read_csv('compas-scores-two-years.csv')
display(df.head())

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
3,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0
4,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,...,1,Low,2013-03-26,NaN,NaN,2,0,1102,0,0


In [ ]:
df.columns

Index(['id', 'name', 'first', 'last', 'compas_screening_date', 'sex', 'dob',
       'age', 'age_cat', 'race', 'juv_fel_count', 'decile_score',
       'juv_misd_count', 'juv_other_count', 'priors_count',
       'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 'c_case_number',
       'c_offense_date', 'c_arrest_date', 'c_days_from_compas',
       'c_charge_degree', 'c_charge_desc', 'is_recid', 'r_case_number',
       'r_charge_degree', 'r_days_from_arrest', 'r_offense_date',
       'r_charge_desc', 'r_jail_in', 'r_jail_out', 'violent_recid',
       'is_violent_recid', 'vr_case_number', 'vr_charge_degree',
       'vr_offense_date', 'vr_charge_desc', 'type_of_assessment',
       'decile_score.1', 'score_text', 'screening_date',
       'v_type_of_assessment', 'v_decile_score', 'v_score_text',
       'v_screening_date', 'in_custody', 'out_custody', 'priors_count.1',
       'start', 'end', 'event', 'two_year_recid'],
      dtype='object')

In [ ]:
df.columns
# Keep only the needed columns
df = df[df['race'].isin(['African-American', 'Caucasian'])]
df = df.dropna(subset=['age', 'priors_count', 'c_charge_degree', 'two_year_recid'])

In [ ]:
# One-hot encode the categorical column
df = pd.get_dummies(df, columns=['c_charge_degree'], drop_first=True)

In [ ]:
# Select all required columns
features = ['age', 'priors_count'] + [col for col in df.columns if 'c_charge_degree' in col]
X = df[features]
y = df['two_year_recid'].astype(int)
race = df['race']

In [ ]:
X_train, X_test, y_train, y_test, race_train, race_test = \
    train_test_split(X, y, race, test_size=0.3, stratify=race)

In [ ]:
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
from fairlearn.metrics import (
    MetricFrame,
    true_positive_rate,
    false_positive_rate,
    selection_rate,
    demographic_parity_difference,
    equalized_odds_difference,
    false_negative_rate,
    true_negative_rate
)

In [ ]:
metric_frame = MetricFrame(
    metrics={
        'TPR': true_positive_rate,
        'FPR': false_positive_rate,
        'FNR': false_negative_rate,
        'Selection Rate': selection_rate
    },
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=race_test
)

print("Fairness Metrics by Race Group:\n", metric_frame.by_group)

Fairness Metrics by Race Group:
                        TPR       FPR       FNR  Selection Rate
race                                                          
African-American  0.653641  0.333333  0.346359        0.495942
Caucasian         0.406780  0.140590  0.593220        0.247283


In [ ]:
tpr_gap = abs(0.665 - 0.481)  # ~0.184
fpr_gap = abs(0.342 - 0.146)  # ~0.196
selection_gap = abs(0.505 - 0.274)  # ~0.231

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
from fairlearn.reductions import ExponentiatedGradient, DemographicParity
from fairlearn.metrics import MetricFrame, selection_rate, true_positive_rate, false_positive_rate, false_negative_rate

In [ ]:
df = pd.read_csv("compas-scores-two-years.csv")

In [ ]:
# Filter to African-American and Caucasian only
df = df[df['race'].isin(['African-American', 'Caucasian'])]
df = df.dropna(subset=['age', 'priors_count', 'r_charge_degree', 'two_year_recid'])

In [ ]:
# One-hot encode categorical features
df = pd.get_dummies(df, columns=['r_charge_degree'], drop_first=True)

In [ ]:
# Prepare features and labels
features = ['age', 'priors_count'] + [col for col in df.columns if 'r_charge_degree' in col]
X = df[features]
y = df['two_year_recid'].astype(int)
sensitive = df['race'].map({'Caucasian': 0, 'African-American': 1})

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test, s_train, s_test = train_test_split(
    X, y, sensitive, test_size=0.3, random_state=42, stratify=y
)